<a href="https://colab.research.google.com/github/kauravs1222/Face-Mask-Detector/blob/master/Mask_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mask Detection Model For Single Image**

- Importing Libraries

In [0]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
import cv2
import matplotlib.pyplot as plt

- Download Pre-Trained Model and Haarcascades File

In [0]:
!wget https://github.com/kauravs1222/Face-Mask-Detector/blob/master/keras_model.h5
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

- Load Pre-trained Model

In [0]:
# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('/content/keras_model.h5', compile=False)

- Upload image to work with

In [0]:
from google.colab import files
file=files.upload()

- Take Image and Converts it

In [0]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

#Change path with your file path
image_path = '/content/images (1).jpg'

# Replace this with the path to your image
image = Image.open(image_path)

normal = plt.imshow(image)

#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# display the resized image
image.show()

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

- Predict the Image and Define Label

In [0]:
# run the inference
prediction = model.predict(data)

prediction = prediction.argmax(axis=-1)
#print(prediction)

label = "Mask On" if prediction == 0 else "No Mask"
color = (0, 255, 0) if prediction == 0 else (255, 0, 0)

- Detect Face and make box

In [0]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#Creating Box for face
if(len(faces)!=0):
  for (x,y,w,h) in faces:
      cv2.rectangle(im_rgb,(x,y),(x+w,y+h),color, 4)

  cv2.rectangle(im_rgb, (x,y), (x+w,y-35), color, cv2.FILLED)
  cv2.putText(im_rgb, label, (x+5, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2, cv2.LINE_AA)
   
else:
  cv2.rectangle(im_rgb, (5,37), (140,5), color, cv2.FILLED)
  cv2.putText(im_rgb, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2, cv2.LINE_AA)

detect = plt.imshow(im_rgb)